In [15]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import pyxem as pxm
import sys
import cv2
from tqdm import tqdm
from scipy.spatial import distance, distance_matrix
import random as rd

import matplotlib 
%matplotlib inline

In [ ]:
########################## build perspective matrix list for all images for drift&distortion correction
def get_perspective_matrix(img1, img2, MIN_MATCH_COUNT = 10, plot = False):
    # Initiate SIFT detector
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        h,w = img1.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
        matchesMask = None
    if plot == True:
        plt.figure()
        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                           singlePointColor = None,
                           matchesMask = matchesMask, # draw only inliers
                           flags = 2)
        img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
        plt.imshow(img3, 'gray'),plt.show()
    return M

def matrix_shift_after_perspective(shift_x, shift_y, perspective_matrix):
    translation_matrix = np.float32([[1,0,shift_y], [0,1,shift_x], [0,0,1]])
    return np.dot(translation_matrix,M)

############################## find warp matrix for each image, apply to each images

perspective_0 = np.float32(np.identity(3)) 
perspective_matrixes = [perspective_0]
for num_mapping in range(8):
    img1 = cv2.imread(f'data/index_mapping_confid_origin_{num_mapping}.jpeg',0)[:,:]          # queryImage
    img2 = cv2.imread(f'data/index_mapping_confid_origin_{num_mapping+1}.jpeg',0)[:,:]          # trainImage
    M = get_perspective_matrix(img2, img1)
    perspective_matrixes.append(M)
    
for i in range(1, len(perspective_matrixes)):
    perspective_matrixes[i] = np.dot(perspective_matrixes[i-1],perspective_matrixes[i])

############################## save warped confid images in a stack

image=[]
for num_mapping in range(8):
    img = cv2.imread(f'data/index_mapping_confid_origin_{num_mapping}.jpeg',0)
    if num_mapping == 3:
        img = cv2.copyMakeBorder(img, 0,100,0,100, cv2.BORDER_CONSTANT, value = 0)
    else:
        img = cv2.warpPerspective(img, perspective_matrixes[num_mapping], (500,500))
    image.append(img)
image = np.array(image)
tif.imsave(f'data/index_mapping_confid_warp.tif', image)

################################  perspective matrix list save in perspective_matrixes
perspective_matrixes = np.array(perspective_matrixes)
np.save(f'data/perspective_matrixes.npy', perspective_matrixes)